# This is a sample Jupyter Notebook

Below is an example of a code cell. 
Put your cursor into the cell and press Shift+Enter to execute it and select the next one, or click 'Run Cell' button.

Press Double Shift to search everywhere for classes, files, tool windows, actions, and settings.

To learn more about Jupyter Notebooks in PyCharm, see [help](https://www.jetbrains.com/help/pycharm/ipython-notebook-support.html).
For an overview of PyCharm, go to Help -> Learn IDE features or refer to [our documentation](https://www.jetbrains.com/help/pycharm/getting-started.html).

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import warnings
from konlpy.tag import Komoran

In [2]:
file_path = "data/scoring_data.xlsx"
# 시트 목록 확인
excel_file = pd.ExcelFile(file_path)
print("사용 가능한 시트 목록:")
print(excel_file.sheet_names)

# 각 시트별로 DataFrame 생성
dataframes = {}

for sheet in excel_file.sheet_names:
    # 데이터 읽기
    df = pd.read_excel(
        file_path,
        sheet_name=sheet,
        header=0,
        usecols=[5,8]
    )

    # 변수명으로 저장 (sheet 이름으로 DataFrame 생성)
    dataframes[sheet] = df

    # 각 DataFrame 정보 출력
    print(f"\n=== {sheet} DataFrame ===")
    print(f"데이터 크기: {df.shape}")
    print(f"열 이름: {df.columns.tolist()}")
    print("\n미리보기:")
    print(df.head())
    print("\n결측치 현황:")
    print(df.isnull().sum())
    print("-" * 50)

사용 가능한 시트 목록:
['Sheet1', 'Sheet2', 'Sheet3', 'Sheet4', 'Sheet5', 'Sheet6', 'Sheet7', 'Sheet8', 'Sheet9', 'Sheet10', 'Sheet11', 'Sheet12', 'Sheet13', 'Sheet14', 'Sheet15', 'Sheet16', 'Sheet17', 'Sheet18', 'Sheet19', 'Sheet20', 'Sheet21', 'Sheet22', 'Sheet23', 'Sheet24', 'Sheet25', 'Sheet26', 'Sheet27', 'Sheet28', 'Sheet29', 'Sheet30', 'Sheet31', 'Sheet32', 'Sheet33', 'Sheet34', 'Sheet35', 'Sheet36', 'Sheet37', 'Sheet38', 'Sheet39', 'Sheet40', 'Sheet41', 'Sheet42', 'Sheet43', 'Sheet44', 'Sheet45', 'Sheet46', 'Sheet47', 'Sheet48', 'Sheet49', 'Sheet50']

=== Sheet1 DataFrame ===
데이터 크기: (213, 2)
열 이름: ['원소 스펙트럼은 선 스펙트럼으로 나타나며 원소마다 선의 위치, 개수, 굵기가 다릅니다. 원소마다 다른 선 스펙트럼이 나타나는 이유는 무엇인가요?', '하현점수']

미리보기:
  원소 스펙트럼은 선 스펙트럼으로 나타나며 원소마다 선의 위치, 개수, 굵기가 다릅니다. 원소마다 다른 선 스펙트럼이 나타나는 이유는 무엇인가요?  \
0                                 원소마다 내는 빛이 다르기 때문에                                 
1    전자들이 높은 에너지상태에서 낮은 에너지상태로 떨어지면서 특정파장의 빛을 방출하기때문                                 
2                          원소마다 가지고 있

In [3]:
komoran = Komoran()

for sheet, df in dataframes.items():
    # 첫 번째 컬럼의 값을 안전하게 Komoran에 적용
    def tokenize_safe(text):
        # 입력값이 None, NaN, 빈 문자열인지 확인 -> 안전한 값만 Komoran 호출
        try:
            return komoran.morphs(str(text))
        except Exception as e:
            print(f"Error: {e} value: {text}")
            return []

    # 새로운 컬럼 "토큰화_결과" 추가 (안전한 토큰화 함수 적용)
    df['토큰화_결과'] = df.iloc[:, 0].apply(tokenize_safe)

    # 데이터프레임 결과 확인
    print(f"\n=== {sheet} DataFrame (토큰화 결과 추가) ===")
    print(df)


Error: java.lang.NullPointerException: Cannot invoke "java.lang.Integer.intValue()" because the return value of "kr.co.shineware.util.common.model.Pair.getFirst()" is null value:  
Error: java.lang.NullPointerException: Cannot invoke "java.lang.Integer.intValue()" because the return value of "kr.co.shineware.util.common.model.Pair.getFirst()" is null value:  
Error: 'utf-8' codec can't decode byte 0xed in position 0: invalid continuation byte value: 🤔

=== Sheet1 DataFrame (토큰화 결과 추가) ===
    원소 스펙트럼은 선 스펙트럼으로 나타나며 원소마다 선의 위치, 개수, 굵기가 다릅니다. 원소마다 다른 선 스펙트럼이 나타나는 이유는 무엇인가요?  \
0                                   원소마다 내는 빛이 다르기 때문에                                 
1      전자들이 높은 에너지상태에서 낮은 에너지상태로 떨어지면서 특정파장의 빛을 방출하기때문                                 
2                            원소마다 가지고 있는 물질이 다르기 때문이다.                                 
3                               원소의 종류와 온도가 다 다르기 때문이다                                 
4                              원소의 종류나 온도가 각각 다르기 때문이다            

In [5]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from collections import Counter

# Iterate over all sheets and process
for sheet, df in dataframes.items():
    # Preparing data
    text_data = df['토큰화_결과'].apply(lambda x: ' '.join(x))  # Convert tokenized lists to strings
    scores = df['하현점수']  # Target labels

    # Check distribution
    score_counts = Counter(scores)
    print(f"Score distribution in {sheet}: {score_counts}")

    # Vectorize text data
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(text_data)
    y = scores.fillna(0)

    # Split dataset (stratified to account for class imbalance)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42, stratify=y
    )

    # Train classifier
    clf = RandomForestClassifier(random_state=42, class_weight="balanced")
    clf.fit(X_train, y_train)

    # Evaluate model
    y_pred = clf.predict(X_test)
    print(f"\n=== Results for {sheet} ===")
    print("Classification Report:\n", classification_report(y_test, y_pred))
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


Score distribution in Sheet1: Counter({0: 169, 2: 24, 1: 20})

=== Results for Sheet1 ===
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.85      0.92        34
           1       0.17      0.25      0.20         4
           2       0.62      1.00      0.77         5

    accuracy                           0.81        43
   macro avg       0.60      0.70      0.63        43
weighted avg       0.88      0.81      0.84        43

Confusion Matrix:
 [[29  5  0]
 [ 0  1  3]
 [ 0  0  5]]
Score distribution in Sheet2: Counter({0: 156, 1: 37, 2: 20})

=== Results for Sheet2 ===
Classification Report:
               precision    recall  f1-score   support

           0       0.94      1.00      0.97        32
           1       1.00      0.71      0.83         7
           2       1.00      1.00      1.00         4

    accuracy                           0.95        43
   macro avg       0.98      0.90      0.93        43
weighted 

/Users/sangyeop/miniconda3/envs/Scoring/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/sangyeop/miniconda3/envs/Scoring/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/sangyeop/miniconda3/envs/Scoring/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{m


=== Results for Sheet4 ===
Classification Report:
               precision    recall  f1-score   support

           0       0.92      1.00      0.96        12
           1       1.00      0.75      0.86        12
           2       0.90      1.00      0.95        19

    accuracy                           0.93        43
   macro avg       0.94      0.92      0.92        43
weighted avg       0.94      0.93      0.93        43

Confusion Matrix:
 [[12  0  0]
 [ 1  9  2]
 [ 0  0 19]]
Score distribution in Sheet5: Counter({0: 125, 2: 78, 1: 10})

=== Results for Sheet5 ===
Classification Report:
               precision    recall  f1-score   support

           0       0.93      1.00      0.96        25
           1       0.00      0.00      0.00         2
           2       1.00      0.94      0.97        16

    accuracy                           0.93        43
   macro avg       0.64      0.65      0.64        43
weighted avg       0.91      0.93      0.92        43

Confusion Matrix

/Users/sangyeop/miniconda3/envs/Scoring/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/sangyeop/miniconda3/envs/Scoring/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/sangyeop/miniconda3/envs/Scoring/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{m


=== Results for Sheet8 ===
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.92      0.96        13
           2       0.97      1.00      0.98        30

    accuracy                           0.98        43
   macro avg       0.98      0.96      0.97        43
weighted avg       0.98      0.98      0.98        43

Confusion Matrix:
 [[12  1]
 [ 0 30]]
Score distribution in Sheet9: Counter({0: 84, 2: 67, 1: 62})

=== Results for Sheet9 ===
Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.88      0.94        17
           1       0.85      0.92      0.88        12
           2       0.87      0.93      0.90        14

    accuracy                           0.91        43
   macro avg       0.90      0.91      0.90        43
weighted avg       0.91      0.91      0.91        43

Confusion Matrix:
 [[15  1  1]
 [ 0 11  1]
 [ 0  1 13]]
Score distribution in Sheet10: Co

/Users/sangyeop/miniconda3/envs/Scoring/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/sangyeop/miniconda3/envs/Scoring/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/sangyeop/miniconda3/envs/Scoring/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{m


=== Results for Sheet44 ===
Classification Report:
               precision    recall  f1-score   support

           0       0.67      0.46      0.55        13
           1       0.22      0.67      0.33         3
           2       1.00      0.33      0.50         3

    accuracy                           0.47        19
   macro avg       0.63      0.49      0.46        19
weighted avg       0.65      0.47      0.50        19

Confusion Matrix:
 [[6 7 0]
 [1 2 0]
 [2 0 1]]
Score distribution in Sheet45: Counter({2: 37, 0: 30, 1: 27})

=== Results for Sheet45 ===
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         6
           1       1.00      1.00      1.00         5
           2       1.00      1.00      1.00         8

    accuracy                           1.00        19
   macro avg       1.00      1.00      1.00        19
weighted avg       1.00      1.00      1.00        19

Confusion Matrix:
 [[6 

ValueError: The least populated class in y has only 1 member, which is too few. The minimum number of groups for any class cannot be less than 2.